In [66]:
!pip install -qU langchain-Groq

In [67]:
#Biblioteca do colab para obter a chave token
from google.colab import userdata

In [68]:
import os #vai ajudar a pegar o token
from langchain.prompts import PromptTemplate #biblioteca do template que vou fazer
from langchain_groq import ChatGroq #site que vou usar para pegar os modelos
from langchain.chains import LLMChain # função para conectar os componentes(Chains)

In [69]:
#Pegando meu token e escolhendo o modelo que vou utilizar

os.environ["GROQ_API_KEY"] = userdata.get("Croq")
meu_modelo = ChatGroq(model="llama3-8b-8192", temperature = 0)

# **Criando um armazenamento para o meu chat**

In [70]:
from langchain_core.chat_history import(
    BaseChatMessageHistory,
    InMemoryChatMessageHistory
)
from langchain_core.runnables.history import RunnableWithMessageHistory

#O banco de mensagens é um dicionário que armazena o conteúdo das conversas
 # atrelados a um id(que representa uma sessão de conversa)

banco_mensagens= {} # É aqui que o histórico de conversas vai ficar

def pegar_historico_conversas(session_id: str) -> BaseChatMessageHistory:
  if session_id not in banco_mensagens:
    banco_mensagens[session_id] = InMemoryChatMessageHistory()
  return banco_mensagens[session_id]

#Juntando meu modelo com a função de armazenamento
with_message_history= RunnableWithMessageHistory(meu_modelo, pegar_historico_conversas )


In [71]:
#Aqui testo a capacidade de memorização

from langchain_core.messages import HumanMessage

with_message_history.invoke(
    HumanMessage(content="Olá, meu nome é Marcus e eu adoro comer açaí"),
    config= {"configurable": {"session_id": "1"}} #Aqui passo o id da conversa. Sera armazenada com key "1" dentro do dicionário de memória
)

AIMessage(content='Olá Marcus! Prazer em conhecê-lo! Ah, você é um fã do açaí, é? É um fruto delicioso e muito saudável, não é? Qual é o seu modo preferido de consumir açaí? Você gosta de tomar smoothie, fazer açaí bowl, ou talvez usar açaí como ingrediente em receitas culinárias?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 25, 'total_tokens': 113, 'completion_time': 0.067588772, 'prompt_time': 0.006323869, 'queue_time': 0.11530298, 'total_time': 0.073912641}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_8dc6ecaf8e', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d398f3bc-43d4-4d35-adf8-5eaeabdebc16-0', usage_metadata={'input_tokens': 25, 'output_tokens': 88, 'total_tokens': 113})

In [72]:
with_message_history.invoke(
    HumanMessage(content="Olá, você lemnbra do meu nome e do que eu adoro?"),
    config = {"configurable": {"session_id": "1"}}
)

#Aqui vejo que ele se lembrou do que eu falei pra ele antes

AIMessage(content='Sim! Eu me lembro perfeitamente! Você se chama Marcus e você adora comer açaí!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 139, 'total_tokens': 165, 'completion_time': 0.020016451, 'prompt_time': 0.016882826, 'queue_time': 0.680710709, 'total_time': 0.036899277}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_8dc6ecaf8e', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--29f97dc9-14d2-4928-8bd2-9e0820db8dba-0', usage_metadata={'input_tokens': 139, 'output_tokens': 26, 'total_tokens': 165})

In [73]:
#Aqui faço um simples propmt para meu chatbot, nada que eu não tenha visto antes
#Após faze-lo eu acorrento(faço uma chain) com ele e meu modelo

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "Você é um assistente, responda o melhor que puder."
         ),MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | meu_modelo

In [74]:
resposta = chain.invoke({"messages": [HumanMessage(content="Olá, sou Pablo")]})

resposta.content

'Olá Pablo! Prazer em conhecê-lo! Estou aqui para ajudá-lo com qualquer coisa que precise. Qual é o seu objetivo ou o que você gostaria de discutir?'

In [75]:
with_message_history= RunnableWithMessageHistory(chain, pegar_historico_conversas)

In [76]:
resposta2= with_message_history.invoke(
    HumanMessage(content= "Olá, meu nome é Pablo. Gosto de açucar"),
    config= {"configurable": {"session_id": "2"}}
)

resposta2.content

'Olá Pablo! Prazer em conhecê-lo! Ah, você gosta de açúcar, é? Qual é o seu tipo de açúcar favorito? Você prefere o doce, o salgado ou algo mais exótico?'

In [77]:
resposta2= with_message_history.invoke(
    [HumanMessage(content="Olá, quem sou eu?")],
    config= {"configurable": {"session_id": "2"}}
)

resposta2.content

'Olá Pablo! Você é você mesmo! Eu sou o seu assistente, aqui para ajudá-lo com qualquer coisa que precise.'

In [78]:
#Testando prompt novamente, mas dessa vez com variáveis. Aqui coloquei linguagem

prompt = ChatPromptTemplate.from_messages(
  [
    (
        "system",
        "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
    ),
    MessagesPlaceholder(variable_name= "messages"),
  ]
)

chain = prompt | meu_modelo

In [79]:
#Setei como Coreano

resposta2 = chain.invoke(
    {"messages": [HumanMessage(content="Olá, sou Arthur!")], "language":"Coreano"}
)

resposta2.content

'Annyeonghaseyo! (안녕하세요!) Nice to meet you, Arthur!'

In [80]:
with_message_history = RunnableWithMessageHistory(
    chain,
    pegar_historico_conversas,
    input_messages_key="messages"
)

In [81]:
resposta3= with_message_history.invoke(
    {"messages": [HumanMessage(content="Hello, how are you feeling today?")], "language":"Coreano"},
    config= {"configurable": {"session_id": "3"}}
)

resposta3.content

"! (annyeonghaseyo! jeoneun joheunhamnida) - Hello! I'm doing well, thank you for asking. How about you?"

# **Managing Conversation History**

In [82]:
#Aqui posso definir meu próprio histórico e setar alguns parâmetros

from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.messages import AIMessage

#Aqui defino alguns parâmetros
trimmer = trim_messages(
    max_tokens=65,
    strategy= "last",
    token_counter= meu_modelo,
    include_system= True,
    allow_partial= False,
    start_on= "human",
)

#A partir desse histórico consigo dar mais conhecimento para meu bot.
messages = [
    SystemMessage(content="Você é um assistente"),
    HumanMessage(content="Olá, sou Marcus"),
    AIMessage(content="olá!"),
    HumanMessage(content="Eu gosto de sorvete de açaí"),
    AIMessage(content="bacana"),
    HumanMessage(content="quanto é 2 + 2?"),
    AIMessage(content="4"),
    HumanMessage(content="obrigado"),
    AIMessage(content="sem problemas"),
]

trimmer.invoke(messages)

[SystemMessage(content='Você é um assistente', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Olá, sou Marcus', additional_kwargs={}, response_metadata={}),
 AIMessage(content='olá!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Eu gosto de sorvete de açaí', additional_kwargs={}, response_metadata={}),
 AIMessage(content='bacana', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='quanto é 2 + 2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='obrigado', additional_kwargs={}, response_metadata={}),
 AIMessage(content='sem problemas', additional_kwargs={}, response_metadata={})]

In [83]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough


#Acorrentando o prompt, modelo e o trimmer que eu criei
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") |trimmer)
    | prompt
    | meu_modelo
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="Qual é meu nome?")],
        "language": "spanish",

    }
)

response.content

'Lo siento, pero como soy un asistente artificial, no tengo información sobre tu nombre. ¿Quieres decirme quién eres?'

In [84]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked: "quanto é 2 + 2?" which is Portuguese for "what is 2 + 2?"'